In [1]:
%matplotlib inline
from __future__ import division
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import bokeh.plotting as bplt
from bokeh.models import ColumnDataSource
from bokeh.models.tools import HoverTool
from bokeh.layouts import gridplot,row,column
from bokeh.io import output_notebook
from util.DaskProfileReader import *
from bokeh.charts import BoxPlot,Bar,Line,TimeSeries
from bokeh.models import Range1d, FixedTicker
from bokeh.models import CheckboxGroup, CustomJS

from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))
output_notebook()

Loading BokehJS ...

In [2]:
rmsd300 = pd.DataFrame(columns=['Block','Try','Time'])
rmsd300everything = pd.DataFrame(columns=['action','alpha','color','duration','key', 'name','start','y','Run'])
block_size = [144]

for block in block_size:
    for i in range(1,40):
        temp = pd.DataFrame(columns=['Block','Try','Task','Time'])
        data1 = DaskProfileReader('stragglers/iteration2/4node_stat/stragglers_test_300_%d_%d.txt'%(block,i))
        temp['Time']=data1[data1['name']=='block_rmsd']['duration'].values/1000
        temp['Try']='Run %d'%i
        data1['Run']='Run %d'%i
        temp['Block']=block
        rmsd300 = rmsd300.append(temp)
        rmsd300everything=rmsd300everything.append(data1)
    
rmsd300=rmsd300.reset_index().drop('index',axis=1)
rmsd300everything=rmsd300everything.reset_index().drop('index',axis=1)

In [3]:
def checkboxesFun(bar):
    group = "Try"
    checkboxes = CheckboxGroup(labels=np.unique(df[group]).tolist(),
                           # Make all checkboxes checked by default
                           active=list(range(np.unique(df[group]).size)))
    checkboxes.callback = CustomJS(args=dict(bars=bar), code="""

    var group = '%s';

    function change_bars_visibility(checkbox_name, visible) {
        for (j = 0; j < bars.renderers.length; j++) {
            // Go through rendered objects

            if (bars.renderers[j].attributes.hasOwnProperty('data_source') && 
                bars.renderers[j].data_source.data[group][0] === checkbox_name) {

                // Change the visibility of this rendered object if it belongs to
                // the group determined by the checkbox that was clicked
                bars.renderers[j].visible = visible;
            }
        }
    }

    for (i = 0; i < cb_obj.labels.length; i++) {
        // Go through checkbox labels
        var checkbox_name = cb_obj.labels[i];

        if (cb_obj.active.indexOf(i) >= 0) {
            // alert(checkbox_name + " is activated");
            change_bars_visibility(checkbox_name, true);
        }
        else {
            // alert(checkbox_name + " is disabled");
            change_bars_visibility(checkbox_name, false);
        }
    }
    """ % group)
    return checkboxes

In [4]:
block_size = [144]
for block in block_size:
    df = rmsd300[rmsd300['Block']==block]
    p2 = BoxPlot(df, values='Time', label=['Try'],
            title="Dask RMSD task execution time for x100 and %d tasks for 40 runs"%block,plot_width=1500,plot_height=800,legend=False)
    
    bplt.show(p2)

In [5]:
block_rmsd_timings = np.load('stragglers/iteration2/4node_stat/timings300.npy')
hist,edges = np.histogram(block_rmsd_timings,1000)
mean = block_rmsd_timings.mean()
std = block_rmsd_timings.std()
hist2,edges2 = np.histogram(df['Time'].values,100)
hist_plot=bplt.figure(height=400,width=700,x_axis_label='Block RMSD Execution Time',title='Block RMSD x100 Dask Task samples μ=%f, σ=%f'%(mean,std))
hist_plot.quad(top=hist, bottom=0, left=edges[:-1], right=edges[1:],fill_color="#036564", line_color="#033649")
hist_plot2=bplt.figure(height=400,width=700,x_axis_label='Block RMSD Dask Task Execution Time',title='Block RMSD x100 100K samples μ=%f, σ=%f'%(df['Time'].mean(),df['Time'].std()))
hist_plot2.quad(top=hist2, bottom=0, left=edges2[:-1], right=edges2[1:],fill_color="#036564", line_color="#033649")
bplt.show(row(hist_plot,hist_plot2))
print 'Function executions over 2 sigma from the mean: ',np.count_nonzero(block_rmsd_timings>(mean+2*std))
print 'Dask Tasks over 2 sigma from the mean: ',np.count_nonzero(df['Time'].values>(df['Time'].mean()+2*df['Time'].std()))

Function executions over 2 sigma from the mean:  7241
Dask Tasks over 2 sigma from the mean:  0


# Number of Compute Tasks vs Number of Transfers

In [6]:
task_types = pd.DataFrame(columns=['transfer','compute','disk-read'])
for i in range(1,40):
    sample = rmsd300everything[rmsd100everything['Run']=='Run %d'%i]
    a = sample['action'].value_counts()
    task_types.loc[len(task_types)] = a.values.tolist() if a.values.size==3 else a.values.tolist()+[0]
task_types.plot(kind='bar',figsize=(20,10))

NameError: name 'rmsd100everything' is not defined

# Delays between Compute Tasks

In [ ]:
delays = list()
for i in range(1,40):
    sample = rmsd300everything[(rmsd100everything['Run']=='Run %d'%i) & (rmsd100everything['action']=='compute')]
    for proc_id in sample['y'].unique():
        proc_tasks = sample[sample['y']==proc_id].reset_index(drop=True)
        times = list()
        for index, row in proc_tasks.iterrows():
            if index ==0:
                start,finish = row['start'],row['start']+row['duration']
            else:
                start = row['start']
                times.append(start - finish)
                finish = row['start']+row['duration']
            
        delays +=times

hist,edges = np.histogram(delays,200)
hist_plot=bplt.figure(height=400,width=700,x_axis_label='Delay between Tasks in msec',title='Block RMSD x100 Dask Task delays μ=%f, σ=%f'%(np.mean(delays),np.std(delays)))
hist_plot.quad(top=hist, bottom=0, left=edges[:-1], right=edges[1:],fill_color="#036564", line_color="#033649")

bplt.show(hist_plot)
print 'Dask Tasks Delays over 2 sigma from the mean: ',np.count_nonzero(np.array(delays)>(np.mean(delays)+2*np.std(delays))),'Total Number: ', len(delays)
        

# Scheduler Delays

In [ ]:
init_sched = list()
for i in range(1,40):
    sample = rmsd300everything[(rmsd100everything['Run']=='Run %d'%i) & (rmsd100everything['name']=='block_rmsd')]
    sample['dt']=sample['start']-sample['start'].min()
    init_sched += sample['dt'].values.tolist()
    
hist,edges = np.histogram(init_sched,500)
hist_plot=bplt.figure(height=600,width=800,x_axis_label='Block RMSD Scheduler Delays in msec',title='Block RMSD x100 Dask Task Scheduler Delay μ=%f, σ=%f'%(np.mean(init_sched),np.std(init_sched)))
hist_plot.quad(top=hist, bottom=0, left=edges[:-1], right=edges[1:],fill_color="#036564", line_color="#033649")

bplt.show(hist_plot)
print 'Dask Tasks Scheduler Delays over 2 sigma from the mean: ',np.count_nonzero(np.array(init_sched)>(np.mean(init_sched)+2*np.std(init_sched))),'Total Number: ', len(init_sched)

# Tranfser Times

In [ ]:
tranfer_durations = list()
for i in range(1,40):
    sample = rmsd300everything[(rmsd100everything['Run']=='Run %d'%i) & (rmsd100everything['action']=='transfer')]
    tranfer_durations+=sample['duration'].values.tolist()
        
hist,edges = np.histogram(tranfer_durations,500)
hist_plot=bplt.figure(height=600,width=800,x_axis_label='Block RMSD Transfer Durations in msec',title='Block RMSD x100 Dask Transfer Task Durations μ=%f, σ=%f'%(np.mean(tranfer_durations),np.std(tranfer_durations)))
hist_plot.quad(top=hist, bottom=0, left=edges[:-1], right=edges[1:],fill_color="#036564", line_color="#033649")

bplt.show(hist_plot)
print 'Dask Tasks Trnasfer Durations over 2 sigma from the mean: ',np.count_nonzero(np.array(tranfer_durations)>(np.mean(init_sched)+2*np.std(tranfer_durations))),'Total Number: ', len(tranfer_durations)